In [3]:
import tqdm as notebook_tqdm
import torch
import torch.nn as nn
from typing import Type, Any, Callable, Union, List, Dict, Optional, cast
from torch import Tensor
from collections import OrderedDict 
from torchvision.models.resnet import *
from torchvision.models.resnet import BasicBlock, Bottleneck
from torchvision.models.resnet import ResNet50_Weights

/Users/sid/opt/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/sid/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b
  Referenced from: <08E12B12-6183-307E-BDA0-374FA8EBA2C9> /Users/sid/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <408F81C0-C6C8-33EF-B02E-B683F7178A71> /Users/sid/opt/anaconda3/lib/python3.9/site-packages/torch/lib/libc10.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [59]:
import numpy as np

In [93]:
from PIL import Image
import requests
from io import BytesIO
import torchvision.transforms as transforms

def get_image(path):
    img = Image.open(path)
    img.show()
    resize = transforms.Resize([224, 224])
    img = resize(img)

    to_tensor = transforms.ToTensor()
    img = to_tensor(img)
    conv = torch.nn.Conv2d(4, 3, 1)                                                                                                                                                                             
    y = conv(img) 
    y.unsqueeze_(0)

    #reduce channel size   
    transform = transforms.Lambda(lambda x: x[:3])
    out = transform(img)
    transform2 = torch.nn.Conv2d(3, 1, 1)
    out = transform2(out)
    return y, out

In [36]:
!pip install grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 14.9 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached torch-2.0.1-cp39-none-macosx_10_9_x86_64.whl (143.4 MB)
  Created wheel for grad-cam: filename=grad_cam-1.4.8-py3-none-any.whl size=38243 sha256=2286f09ea639f088a7ad7bdeacc6a8dca7b53c807af81dee3adca432ac446517
  Stored in directory: /Users/sid/Library/Caches/pip/wheels/5b/d7/f5/363ec51afcc6b5dd60bd123bdf06c0189221c6505624b5c35a
Successfully built grad-cam
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1


In [37]:

from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

In [1]:
import os
from os import listdir

images = []

for imge in listdir("/Users/sid/Desktop/MML/CoVA dataset/imgs"):
    images.append(imge)


In [38]:
with open('imagenet_classes.txt') as f:
  classes = [line.strip() for line in f.readlines()]



In [25]:
from torchvision import models
import torch
resnet = models.resnet101(pretrained=True)
resnet.eval()
 


/Users/sid/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/sid/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /Users/sid/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [102]:
target_layers = [resnet.layer4[-1]]

# Construct the CAM object once, and then re-use it on many images:
cam = GradCAM(model=resnet, target_layers=target_layers)

targets = [ClassifierOutputTarget(281)]


In [85]:
transform = transforms.ToPILImage()


In [104]:

for i in range(0, len(images), 50):
            
            img, rgb = get_image("/Users/sid/Desktop/MML/CoVA dataset/imgs/"+images[i])
            
            out = resnet(img)
 
# Forth, print the top 5 classes predicted by the model
            _, indices = torch.sort(out, descending=True)
            percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
            print([(classes[idx], percentage[idx].item()) for idx in indices[0][:5]])
            grayscale_cam = cam(input_tensor=img, targets=None)
            grayscale_cam = grayscale_cam[0, :]
            j = transform(img)
            j.show()
            break
            #print(rgb.shape, grayscale_cam.shape)
            #visualization = show_cam_on_image(img, grayscale_cam, use_rgb=True)
        


[("916: 'web site, website, internet site, site',", 47.16188049316406), ("485: 'CD player',", 5.614315509796143), ("782: 'screen, CRT screen',", 4.761532306671143), ("754: 'radio, wireless',", 3.7328011989593506), ("664: 'monitor',", 3.5938501358032227)]


ValueError: pic should be 2/3 dimensional. Got 4 dimensions.

[[[[0.79726243 2.0561945  1.10494    ... 0.76064336 0.
    0.        ]
   [0.         0.15658307 1.3795116  ... 1.8147564  0.8687253
    0.73710084]
   [0.         0.4587431  0.         ... 0.2987423  0.44863325
    0.        ]
   ...
   [0.5284235  0.         0.39666143 ... 0.         0.29059356
    0.        ]
   [0.9432338  0.3253047  0.15500492 ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.65898776 0.84917694
    0.        ]]

  [[0.69623953 2.2121277  0.01586813 ... 0.         0.
    0.        ]
   [0.56853384 0.9869514  0.         ... 0.         0.26354167
    0.8104534 ]
   [0.38012138 0.         0.         ... 0.58648574 0.68069834
    0.2110058 ]
   ...
   [0.23228824 1.3051324  0.66940963 ... 0.7060129  0.81961155
    0.23807639]
   [0.         0.40454316 0.         ... 0.01432359 1.444592
    1.3261893 ]
   [0.53847194 0.         0.         ... 0.3607704  2.9254348
    2.8094854 ]]

  [[0.         0.27545366 0.         ... 0.         0.45508817

In [24]:
from matplotlib import pyplot as plt

In [ ]:
fig = plt.figure(figsize=(30, 50))
for i in range(len(processed)):
    a = fig.add_subplot(5, 4, i+1)
    imgplot = plt.imshow(processed[i])
    a.axis("off")
    a.set_title(names[i].split('(')[0], fontsize=30)
plt.savefig(str('feature_maps.jpg'), bbox_inches='tight')